In [1]:
import pandas as pd
from zipfile import ZipFile
import requests
import numpy as np
import datetime


csv_url = "https://www150.statcan.gc.ca/n1/pub/13-26-0003/2020001/COVID19-eng.zip"
save_path = 'data.zip'
#data = zip.read(name_of_file_to_read)

In [10]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

download_url(csv_url, save_path)

with ZipFile('data.zip', 'r') as zip:
    zip.extractall()
    print('Done!')

Done!


In [11]:
df = pd.read_csv('COVID19-eng.csv')
df.head()

,COV_ID,COV_REG,COV_EW,COV_EWG,COV_EY,COV_GDR,COV_AGR,COV_OCC,COV_ASM,COV_OW,COV_OY,COV_HSP,COV_RSV,COV_RW,COV_RY,COV_DTH,COV_TRM
0,1,3,16,0,21,2,5,9,9,99,99,3,1,99,99,2,1
1,2,3,43,0,20,2,3,2,2,43,20,9,1,99,99,2,1
2,3,2,49,0,20,2,4,4,2,49,20,3,1,51,20,2,1
3,4,3,25,0,20,2,2,9,2,25,20,3,1,99,99,2,1
4,5,2,49,0,20,1,1,4,1,99,99,3,1,50,20,2,1


In [12]:
#recode numerical data so it is intelligible
df.replace({'COV_REG':{1: 'Atlantic', 2: 'Quebec', 3: 'Ontario and Nunavut', 4: 'Prairies and NWT',
                              5: 'BC and Yukon'}}, inplace = True)
df.replace({'COV_AGR':{1: '0-19', 2: '20-29', 3: '30-39', 4: '40-49', 5: '50-59', 6: '60-69', 
                                 7: '70-79', 8: '80+', 99: np.nan}}, inplace = True)
df.replace({'COV_ASM':{1: 'Y', 2: 'N', 9: np.nan}}, inplace = True)
df.replace({'COV_GDR':{1: 'M', 2: 'F', 9: np.nan}}, inplace = True)
df.replace({'COV_HSP':{1: 'Y_ICU', 2: 'Y', 3: 'N', 9: np.nan}}, inplace = True)
df.replace({'COV_OCC':{1: 'healthcare', 2: 'school', 3: 'longtermcareresident', 4: 'Other', 
                                9: np.nan}}, inplace = True)
df.replace({'COV_TRM':{1: 'domestic', 2: 'international', 9: None}}, inplace = True)
df.replace({'COV_DTH':{1: 'Y', 2: 'N', 9: np.nan}}, inplace = True)
df.replace({'COV_RSV':{1: 'Y', 2: 'N', 3: np.nan}}, inplace = True)
df.replace({'COV_EW':{99: np.nan}}, inplace = True)
df.replace({'COV_EWG':{99: np.nan}}, inplace = True)
df.replace({'COV_EY':{99: np.nan}}, inplace = True)
df.replace({'COV_OW':{99: np.nan}}, inplace = True)
df.replace({'COV_OY':{99: np.nan}}, inplace = True)
df.replace({'COV_RW':{99: np.nan}}, inplace = True)
df.replace({'COV_RY':{99: np.nan}}, inplace = True)
df.columns = ['ID', 'region', 'week', 'week_group', 'year', 'gender', 'age', 'occupation', 'asymptomatic', 'onset_week', 'onset_year', 'hospitalization', 'resolved', 'resolution_week', 'resolution_year', 'death', 'transmission']
df.head()

,ID,region,week,week_group,year,gender,age,occupation,asymptomatic,onset_week,onset_year,hospitalization,resolved,resolution_week,resolution_year,death,transmission
0,1,Ontario and Nunavut,16.0,0,21.0,F,50-59,NaN,NaN,NaN,NaN,N,Y,NaN,NaN,N,domestic
1,2,Ontario and Nunavut,43.0,0,20.0,F,30-39,school,N,43.0,20.0,NaN,Y,NaN,NaN,N,domestic
2,3,Quebec,49.0,0,20.0,F,40-49,Other,N,49.0,20.0,N,Y,51.0,20.0,N,domestic
3,4,Ontario and Nunavut,25.0,0,20.0,F,20-29,NaN,N,25.0,20.0,N,Y,NaN,NaN,N,domestic
4,5,Quebec,49.0,0,20.0,M,0-19,Other,Y,NaN,NaN,N,Y,50.0,20.0,N,domestic


In [13]:
df["onset"] = "20" + (df.year).astype('Int32').astype(str) + "-W" + (df.week).astype('Int32').astype(str)
#df["onset"] = datetime.datetime.strptime(str(df["onset"]) + '-1', "%Y-W%W-%w")
df["resolution"] = "20" + (df.resolution_year).astype('Int32').astype(str) + "-W" + (df.resolution_week).astype('Int32').astype(str)
#df["resolution"] = datetime.datetime.strptime(str(df["resolution"]) + '-1', "%Y-W%W-%w")
print(df.head())

   ID               region  week  week_group  year gender    age occupation  \
0   1  Ontario and Nunavut  16.0           0  21.0      F  50-59        NaN   
1   2  Ontario and Nunavut  43.0           0  20.0      F  30-39     school   
2   3               Quebec  49.0           0  20.0      F  40-49      Other   
3   4  Ontario and Nunavut  25.0           0  20.0      F  20-29        NaN   
4   5               Quebec  49.0           0  20.0      M   0-19      Other   

  asymptomatic  onset_week  onset_year hospitalization resolved  \
0          NaN         NaN         NaN               N        Y   
1            N        43.0        20.0             NaN        Y   
2            N        49.0        20.0               N        Y   
3            N        25.0        20.0               N        Y   
4            Y         NaN         NaN               N        Y   

   resolution_week  resolution_year death transmission     onset    resolution  
0              NaN              NaN     N

In [14]:
df.loc[df['resolution'].str.contains('NA'), 'resolution'] = np.nan
print(df.head())

   ID               region  week  week_group  year gender    age occupation  \
0   1  Ontario and Nunavut  16.0           0  21.0      F  50-59        NaN   
1   2  Ontario and Nunavut  43.0           0  20.0      F  30-39     school   
2   3               Quebec  49.0           0  20.0      F  40-49      Other   
3   4  Ontario and Nunavut  25.0           0  20.0      F  20-29        NaN   
4   5               Quebec  49.0           0  20.0      M   0-19      Other   

  asymptomatic  onset_week  onset_year hospitalization resolved  \
0          NaN         NaN         NaN               N        Y   
1            N        43.0        20.0             NaN        Y   
2            N        49.0        20.0               N        Y   
3            N        25.0        20.0               N        Y   
4            Y         NaN         NaN               N        Y   

   resolution_week  resolution_year death transmission     onset resolution  
0              NaN              NaN     N   

In [4]:
df.year = (df.year).astype('Int32').astype(str)
print(df.head())

   ID               region  week  week_group year gender    age occupation  \
0   1  Ontario and Nunavut  16.0           0   21      F  50-59        NaN   
1   2  Ontario and Nunavut  43.0           0   20      F  30-39     school   
2   3               Quebec  49.0           0   20      F  40-49      Other   
3   4  Ontario and Nunavut  25.0           0   20      F  20-29        NaN   
4   5               Quebec  49.0           0   20      M   0-19      Other   

  asymptomatic  onset_week  onset_year hospitalization resolved  \
0          NaN         NaN         NaN               N        Y   
1            N        43.0        20.0             NaN        Y   
2            N        49.0        20.0               N        Y   
3            N        25.0        20.0               N        Y   
4            Y         NaN         NaN               N        Y   

   resolution_week  resolution_year death transmission  
0              NaN              NaN     N     domestic  
1             